In [1]:
#!/usr/bin/env python3
import argparse
import sys
import numpy as np
import cv2
import scipy.io as sio
from tensorflow.keras.models import load_model

def load_extraction_params(mat_file_path):
    """
    Optionally load any parameters from your .mat file that are needed
    to perform feature extraction. Adjust this function according to your file's structure.
    """
    mat_contents = sio.loadmat(mat_file_path)
    # For example, if the mat file contains a structure with parameters:
    # params = mat_contents['params']
    # return params
    # If not needed, simply return None.
    return None

def extract_features(image, extraction_params=None):
    """
    Extract 500 features from the image. The code below is a placeholder:
    you must modify it so that it exactly replicates the extraction routine
    (including any smoothing, normalization, etc.) that you applied when building
    your feature matrix from the .mat file.
    """
    # Example using HOG features (adjust parameters as needed)
    # Resize image to a fixed size (ensure that this is consistent with your training extraction)
    resized = cv2.resize(image, (64, 64))
    
    # Initialize HOG descriptor with chosen parameters.
    hog = cv2.HOGDescriptor(_winSize=(64, 64),
                            _blockSize=(16, 16),
                            _blockStride=(8, 8),
                            _cellSize=(8, 8),
                            _nbins=9)
    hog_features = hog.compute(resized)
    hog_features = hog_features.flatten()
    
    # Adjust the length of the feature vector to be exactly 500.
    if hog_features.shape[0] > 500:
        features = hog_features[:500]
    elif hog_features.shape[0] < 500:
        # Pad with zeros if fewer than 500 features are available.
        pad = np.zeros(500 - hog_features.shape[0])
        features = np.concatenate([hog_features, pad])
    else:
        features = hog_features

    return features

def load_pretrained_model(model_path):
    """
    Load the pre-trained model from the provided model file.
    Adjust this function if you are using another framework.
    """
    model = load_model(model_path)
    return model


2025-04-05 21:07:03.982915: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-05 21:07:04.015767: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743867424.052454   99236 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743867424.064363   99236 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743867424.095295   99236 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
def main():
    parser = argparse.ArgumentParser(description="Extract features from a test image and predict its class.")
    parser.add_argument('--image', required=True, help="sketches/airplane/1.png")
    parser.add_argument('--model', required=True, help="best_ann_model.h5")
    parser.add_argument('--params', default="features_shog_smooth.mat", help="feature_extraction/features_shog_smooth.mat")
    args = parser.parse_args()

    # Load extraction parameters if necessary
    extraction_params = load_extraction_params(args.params)

    # Load and validate the test image.
    image = cv2.imread(args.image)
    if image is None:
        print("Error: Could not load image from", args.image)
        sys.exit(1)

    # Extract features from the image.
    features = extract_features(image, extraction_params)
    # Reshape features to match model input shape (assumes model expects a 2D array, e.g., [batch, features])
    features = np.expand_dims(features, axis=0)

    # Load the pre-trained model.
    model = load_pretrained_model(args.model)

    # Predict the class of the image.
    prediction = model.predict(features)
    predicted_class = np.argmax(prediction, axis=1)
    
    print("Predicted class:", predicted_class[0])

if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] --image IMAGE --model MODEL
                             [--params PARAMS]
ipykernel_launcher.py: error: the following arguments are required: --image, --model


SystemExit: 2